In [ ]:
import os
import glob
import trackpy as tp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.optimize import curve_fit
import numpy as np
from matplotlib.ticker import ScalarFormatter

framerate = 4
microns_per_pixel = 0.072  # I think

In [ ]:
file_directory = r"C:\Users\lizau\Desktop\walker_test\final_izbor\2b11\1a8\ROTATED\MSD"
in_files = glob.glob(f'{file_directory}/*.csv')
out_dir = Path(f'{file_directory}\out')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)


In [ ]:
# This cell combines the tray csvs from all measurements into one csv only and makes sure that all the particles have different IDs
trays_to_combine = []
max_particle_id = 0

for file in in_files:
    tray = pd.read_csv(file)
    if 'particle' in tray.columns:         # Offset the particle IDs to ensure uniqueness
        tray['particle'] += max_particle_id
        max_particle_id = tray['particle'].max() + 1
    trays_to_combine.append(tray)
combined_df = pd.concat(trays_to_combine, ignore_index=True)
combined_df = combined_df.sort_values(by='frame').reset_index(drop=True)
combined_df.to_csv(f'{out_dir}/combined_sorted_particles.csv')

print("DataFrames combined successfully!")



In [ ]:
#run the following two lines only if trays are already combined in one csv
data = r"C:\Users\lizau\Desktop\walker_test\RMSD\2b12\1a6_dmng_sucrose\50%\max_lag_50\combined_sorted_particles.csv"
combined_df = pd.read_csv(data)

In [ ]:
# this cell draws IMSD and EMSD plots (alternatively in a log log scale)

tray_len_query = 50
max_lag = 50
coordinate = ['x']

formatter = ScalarFormatter()
formatter.set_scientific(True)
formatter.set_powerlimits((-2, 2))  # controls when sci notation kicks in

combined_df_filtered = combined_df.query(f'length>{tray_len_query}')
im = tp.imsd(combined_df_filtered, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=coordinate)
fig, ax = plt.subplots()
ax.plot(im.index, im, "k-", alpha=0.1)  # black lines, semitransparent
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
yticks = np.linspace(np.round(np.min(im),5), np.round(np.max(im),5), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)
#ax.set_title(f'imsd for trajectories longer than {tray_len_query}')
#ax.set_xscale("log")
#ax.set_yscale("log")
#plt.savefig(f'{out_dir}/combined_trays_{tray_len_query}_maxlag_{max_lag}_imsd_{coordinate}.png')

em = tp.emsd(combined_df_filtered, microns_per_pixel, framerate, max_lagtime=max_lag, pos_columns=coordinate)
fig, ax = plt.subplots()
ax.plot(em.index, em, "o")
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
#ax.set_title(f'emsd for trajectories longer than {tray_len_query}')
yticks = np.linspace(np.round(np.min(em),5), np.round(np.max(em),4), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)
#ax.set_xscale("log")
#ax.set_yscale("log")
#plt.savefig(f'{out_dir}/combined_trays_{tray_len_query}_maxlag_{max_lag}_emsd_{coordinate}.png', bbox_inches='tight')




In [ ]:
#this cell fits a linear funtion to the EMSD graph on log scale
#the slope gives the parameter alpha

def linear_func(x, m, b):
    return m * x + b

linear_start = 0
linear_end = 12

# Extract the specific part of the data for fitting
mask = (em.index > linear_start) & (em.index < linear_end)
x_linear = em.index[mask]
y_linear = em[mask]


# Fit the linear region with curve_fit
popt, pcov = curve_fit(linear_func, np.log10(x_linear), np.log10(y_linear))
slope, intercept = popt

# Plot the original data and the fitted line
fig, ax = plt.subplots()
ax.plot(em.index, em, "o")
ax.plot(x_linear, 10**intercept*x_linear**slope, label=f"Linear Fit: y = {slope:.2e}x + {np.round(intercept,2)}")
ax.set_ylabel(r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", fontsize=14)
ax.set_xlabel("lag time $t$ [s]", fontsize=14)
#ax.set_title(f"Linear Fit for emsd on log/log scale")
ax.set_xscale("log")
ax.set_yscale("log")
yticks = np.linspace(np.round(np.min(em),5), np.round(np.max(em),5), 5)
ax.set_yticks(yticks)
ax.yaxis.get_offset_text().set_fontsize(14)
ax.yaxis.set_major_formatter(formatter)
ax.tick_params(axis='both', labelsize=14)
ax.minorticks_off()
ax.legend(fontsize=12)

plt.savefig(f'{out_dir}/combined_trays_{tray_len_query}_emsd_fit_range_{linear_start}_{linear_end}_{coordinate}.png', bbox_inches='tight')

# Print the fit parameters
print(f"Slope: {slope:.2e}, Intercept: {intercept:.2e}")

In [ ]:
for file in in_files[0:1]:
    print(file)
    basename = os.path.basename(file)
    basename_noext, ext = os.path.splitext(basename)
    data = pd.read_csv(file)
    data = data.query('particle==4')
    im = tp.imsd(data, microns_per_pixel, framerate)
    fig, ax = plt.subplots()
    ax.plot(im.index, im, "k-", alpha=0.1)  # black lines, semitransparent
    ax.set(ylabel=r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", xlabel="lag time $t$")
    #ax.set_xscale("log")
    #ax.set_yscale("log")
    plt.savefig(f'{out_dir}/{basename_noext}_imsd.png')

    em = tp.emsd(data, microns_per_pixel, framerate)
    fig, ax = plt.subplots()
    ax.plot(em.index, em, "o")
    ax.set(ylabel=r"$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]", xlabel="lag time $t$")
    plt.savefig(f'{out_dir}/{basename_noext}_emsd.png', bbox_inches='tight')

    data.plot('x','y')
